# **Example GLAM3 | Impacts of land occupation and transformation on biodiversity**

**Authors:**
[Romain Sacchi](mailto:romain.sacchi@psi.ch)
[Alvaro Hahn Menacho](mailto:alvaro.hahn-menacho@psi.ch)

This notebook presents an example to obtain an exchange-based LCA score using the [Biodiversity impacts](https://pubs.acs.org/doi/full/10.1021/acs.est.3c04191) method developed by Scherrer et al., 2023. The method is implemented in the `edges` package and can be used in `brightway2` and `brightway25` via the `EdgeLCIA` class.

The method, when used, should be cited as follows:

Scherer L, Rosa F, Sun Z, et al (2023)
Biodiversity Impact Assessment Considering Land Use Intensities and Fragmentation.
Environ Sci Technol https://doi.org/10.1021/acs.est.3c04191


## Structure

1. [Import *edges* and set *bw2 project*](#sec_import_edges)
2. [Import hydrogen production, PEM, LCI](#sec_import_h2)
3. [Select *Functional Unit* and *method*](#sec_select_fu)
4. [Run edges based LCA](#sec_run_edges_lca)
5. [Export results](#sec_export_results)

<a id="sec_import_edges"></a>
# 1. Import the ``edges`` and set up a ``brightway`` project

In [1]:
import pandas as pd

from edges import EdgeLCIA, get_available_methods, setup_package_logging
import logging
import bw2data, bw2io

# we want the logger to log things into edges.log
setup_package_logging(level=logging.INFO)
# if you want full debug:
#setup_package_logging(level=logging.DEBUG)

For our example, we activate a ``brighway`` project that contains ecoinvent 3.10 cut-off.

In [2]:
bw2data.projects.set_current("test_severin")
# bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
bw2data.databases

Databases dictionary with 4 object(s):
	biosphere
	biosphere3
	ecoinvent-3.10.1-cutoff
	h2_pem

<a id="sec_import_h2"></a>
# 2. Import hydrogen production, PEM, LCI

To illustrate our water footprint case, we need to import some inventories representing the production of hydrogen using a PEM electrolyzer, powered with offshore wind power in France. The inventories are provided in this repository (``lci-hydrogen-electrolysis-ei310.xlsx``).

In [4]:
if "h2_pem" in bw2data.databases:
    del bw2data.databases["h2_pem"]
lci = bw2io.ExcelImporter("lci-hydrogen-electrolysis-ei310.xlsx")
lci.apply_strategies()
lci.match_database(fields=["name", "reference product", "location"])
lci.match_database("ecoinvent-3.10.1-cutoff",fields=["name", "reference product", "location"])
lci.match_database("biosphere",fields=["name", "categories"])
lci.statistics()
lci.drop_unlinked(i_am_reckless=True)
if len(list(lci.unlinked)) == 0:
    lci.write_database()

# we assign manually classifications to activities
# since it is unclear how to do that in the Excel inventory file.

classifications = {
    "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity": ("CPC", "34210"),
    "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from solar photovoltaic electricity": ("CPC", "34210"),
    "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from onshore wind electricity": ("CPC", "34210"),
    "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from offshore wind electricity": ("CPC", "34210"),
    "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from geothermal electricity": ("CPC", "34210"),
    "electrolyzer production, 1MWe, PEM, Stack": ("CPC", "4220:Construction of utility projects"),
    "treatment of electrolyzer stack, 1MWe, PEM": ("CPC", "3830"),
    "electrolyzer production, 1MWe, PEM, Balance of Plant": ("CPC", "4220:Construction of utility projects"),
    "treatment of electrolyzer balance of plant, 1MWe, PEM": ("CPC", "3830"),
    "platinum group metal, extraction and refinery operations": ("CPC", "2420"),
    "deionized water production, via reverse osmosis, from brackish water": ("CPC", "34210")
}
for ds in bw2data.Database("h2_pem"):
    if ds["name"] in classifications:
        ds["classifications"] = [classifications[ds["name"]]]
        ds.save()


Extracted 1 worksheets in 0.12 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 27.03 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


11 datasets
214 exchanges
0 unlinked exchanges
  
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


0% [###########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/27/2025 18:25:50
  Finished: 12/27/2025 18:25:50
  Total time elapsed: 00:00:00
  CPU %: 97.70
  Memory %: 1.91
Created database: h2_pem


<a id="sec_select_fu"></a>
# 3. Select Functional Unit and method

### 3.1. Select FU
We select the activity to run the LCA for. In our case, it is the production of 1 kg of hydrogen, using a PEM electrolyzer, powered with offshore wind power, in France.

In [5]:
act = [a for a in bw2data.Database("h2_pem") if a["name"] == "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from offshore wind electricity"][0]
act

'hydrogen production, gaseous, 30 bar, from PEM electrolysis, from offshore wind electricity' (kilogram, FR, None)

### 3.2. Select method
``edges`` comes with a numbers of LCIA methods. You can list them using the `get_available_methods()` function.
We currently have several flavors of `GLAM3 biodiversity`:

* 'GLAM3', 'biodiversity', 'occupation', 'average', 'amphibians'
* 'GLAM3', 'biodiversity', 'occupation', 'average', 'birds'
* 'GLAM3', 'biodiversity', 'occupation', 'average', 'eukaryota'
* 'GLAM3', 'biodiversity', 'occupation', 'average', 'mammals'
* 'GLAM3', 'biodiversity', 'occupation', 'average', 'plants'
* 'GLAM3', 'biodiversity', 'occupation', 'average', 'reptiles'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'amphibians'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'birds'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'eukaryota'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'mammals'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'plants'
* 'GLAM3', 'biodiversity', 'transformation', 'average', 'reptiles'


These methods present different scopes:

* Occupation CFs (in PDF·m²⁻¹) quantify impacts per unit area and time of land occupation.
* Transformation CFs (in PDF·yr·m²⁻¹) quantify impacts per unit area transformed, accounting for regeneration time.
* CFs are provided as average, at country level, with ecoregion-specific CFs for sensitivity purpose, across five biome types.
* The CFs integrate land fragmentation (via the Equivalent Connected Area, ECA) and land use intensity—both drivers of biodiversity loss not jointly included in earlier models.
* CFs cannot be used to assess changes in fragmentation degree, as fragmentation is internally parameterized.
* The Eukaryota CFs represent an aggregated proxy combining plants and vertebrates (amphibians, birds, mammals, reptiles), each weighted equally. They thus serve as a generic biodiversity indicator for overall potential species loss across taxa.

For more information about its implementation and use, refer to [the page about](https://edges.readthedocs.io/en/latest/methods.html#glam3-land-use-impacts-on-biodiversity) `GLAM3 biodiversity` in the ``edges`` documentation, and for more information about the method itself, refer to:

Scherer L, Rosa F, Sun Z, et al (2023)
Biodiversity Impact Assessment Considering Land Use Intensities and Fragmentation.
Environ Sci Technol https://doi.org/10.1021/acs.est.3c04191


In [6]:
get_available_methods()

[('AWARE 2.0', 'Country', 'all', 'yearly'),
 ('AWARE 2.0', 'Country', 'irri', 'yearly'),
 ('AWARE 2.0', 'Country', 'non', 'irri', 'yearly'),
 ('AWARE 2.0', 'Country', 'unspecified', 'yearly'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'amphibians'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'birds'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'eukaryota'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'mammals'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'plants'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'reptiles'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'amphibians'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'birds'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'eukaryota'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'mammals'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'plants'),
 ('GLAM3', 'biodiversity', 'transformation', 'average', 'reptiles'),
 ('GeoPol

In [7]:
methods = [
    ('GLAM3', 'biodiversity', 'occupation', 'average', 'amphibians'),
    ('GLAM3', 'biodiversity', 'occupation', 'average', 'birds'),
    # ('GLAM3', 'biodiversity', 'occupation', 'average', 'eukaryota'), <- we remove the eukaryota indicator as it is an aggregate of the other five
    ('GLAM3', 'biodiversity', 'occupation', 'average', 'mammals'),
    ('GLAM3', 'biodiversity', 'occupation', 'average', 'plants'),
    ('GLAM3', 'biodiversity', 'occupation', 'average', 'reptiles'),
]
methods

[('GLAM3', 'biodiversity', 'occupation', 'average', 'amphibians'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'birds'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'mammals'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'plants'),
 ('GLAM3', 'biodiversity', 'occupation', 'average', 'reptiles')]

<a id="sec_run_edges_lca"></a>
# 4. Run edges-based LCA

In [8]:
import pandas as pd

main_df = pd.DataFrame()

for method in methods:
    # we instantiate EdgeLCIA, which builds on the `bw2calc.LCA` class
    LCA = EdgeLCIA(
        demand={act: 1}, # <- we define a functional unit
        method=method, # <- we specify the LCIA method
        use_distributions=True, # <- we want to consider variability across ecoregions
        iterations=10000 # <- we specific the number of iterations
    )

    # apply strategies
    LCA.apply_strategies()
    LCA.evaluate_cfs()
    LCA.lcia()

    df = LCA.generate_cf_table()
    df["method"] = " ".join(method)
    main_df = pd.concat([main_df, df], ignore_index=True)

Processing global groups (pass 1): 100%|██████████| 17/17 [00:00<00:00, 17074.51it/s]


In [ ]:
print(LCA.statistics())

In [ ]:
main_df = main_df.replace("GLAM3 biodiversity occupation average ", "", regex=True)
main_df.to_pickle("LCIA_results/lcia_occupation_results_old.pkl")
print("Dataframe saved successfully!")

Dataframe saved successfully!


Index(['supplier name', 'supplier categories', 'consumer name',
       'consumer reference product', 'consumer location', 'consumer cpc',
       'consumer ecospold01categories', 'consumer isic rev.4 ecoinvent',
       'amount', 'CF (mean)', 'CF (std)', 'CF (min)', 'CF (5th)', 'CF (25th)',
       'CF (50th)', 'CF (75th)', 'CF (95th)', 'CF (max)', 'impact (mean)',
       'impact (std)', 'impact (min)', 'impact (5th)', 'impact (25th)',
       'impact (50th)', 'impact (75th)', 'impact (95th)', 'impact (max)',
       'method'],
      dtype='object')